First, create a 1k rows dataset to work with

In [18]:
# import pandas as pd

# repo_dir="/home/viktor/Documents/kaggle/kaggle_llm"


# input_csv=f"{repo_dir}/data/raw_questions_wiki_sci_2-splitted/more_questions_raw_questions_wiki_sci_2_train.csv"

# df = pd.read_csv(input_csv)

# # take random 1000 rows
# df = df.sample(n=100, random_state=42).reset_index(drop=True)

# df.to_csv("/home/viktor/Documents/kaggle/kaggle_llm/data/wiki-sci-2-w-sentence-context/train_1k.csv", index=False)

# df

,Unnamed: 0,id,prompt,A,B,C,D,E,answer,cluster number,round
0,74899,75238,What is the main advantage of having larger an...,Enhanced physical performance,Greater protection against injury,Higher aesthetic appeal,Improved balance and stability,Increased pleasure during sexual intercourse,B,81,3
1,8449,8508,What is the approximate amplitude of the short...,0.001 magnitudes,0.009 magnitudes,0.005 magnitudes,0.015 magnitudes,0.02 magnitudes,B,177,6
2,91791,92213,What is the commutator in the context of deriv...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a linear endomorphism of an ...,E,970,0
3,8844,8903,What is the main goal of diabetes management a...,To eliminate the need for insulin injections i...,To prevent the development of cardiovascular d...,To avoid acute problems of hyperglycemia or hy...,To maintain blood sugar levels as close to nor...,To restore pancreatic beta cell function in ty...,D,180,7
4,62010,62293,Which of the following statements is true abou...,It is a benign nodular thickening of the vas d...,It is a low-grade adenocarcinoma,It is a complication experienced in approximat...,It is a form of sterilization for males,It is a male genital disorder,C,687,6
...,...,...,...,...,...,...,...,...,...,...,...
95,49205,49446,What is the main condition that Euler system e...,They must be indexed by different number fields.,They must be congruent to each other modulo a ...,They must have special values related to Euler...,They must be elements of a specific Galois coh...,They must be related by a simple formula for d...,E,564,4
96,53893,54150,What is the role of coefficients in a formal p...,To determine the radius of convergence of the ...,To indicate the position of the terms in the s...,To define the function represented by the series.,To represent the value of the variable.,To determine the number of terms in the series.,B,608,5
97,10783,10843,What is the relationship between PSA levels an...,A PSA level of 2.0ng/mL above the nadir is the...,PSA levels should decrease gradually after rad...,PSA levels may continue to decrease for severa...,A rise in PSA levels above the nadir indicates...,PSA levels are not reliable indicators of pros...,D,2,0
98,35123,35273,What is the role of Mn(III) ions produced by MnP?,To regulate the transcriptional activity of Mn...,To stabilize the heme cofactor in the enzyme,To catalyze the oxidation of phenolic compound...,To bind to the Fe(III) ion in the heme cofactor,To facilitate the release of Mn(III)-carboxyli...,C,43,3


In [19]:
# !python $repo_dir/tools/generate_context_wiki_sci_fast.py \
#      --input-csv /home/viktor/Documents/kaggle/kaggle_llm/data/wiki-sci-2-w-sentence-context/train_1k.csv     \
#      --model-dir /home/viktor/Documents/kaggle/kaggle_llm/data/huggingface_hub_models/bge-large-en   \
#      --out-dir /home/viktor/Documents/kaggle/kaggle_llm/data/wiki-sci-2-w-sentence-context    \
#      --out-name "train_1k_w_context"      \
#      -k 70     \
#      --max-context-len 20000     \
#      --wiki-sci-parquets /home/viktor/Documents/kaggle/kaggle_llm/data/wikipedia_pages2_w_embd_sentences \
#      --njobs 4


^C
Traceback (most recent call last):
  File "/home/viktor/Documents/kaggle/kaggle_llm/tools/generate_context_wiki_sci_fast.py", line 6, in <module>
    from sentence_transformers import SentenceTransformer
  File "/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/sentence_transformers/__init__.py", line 3, in <module>
    from .datasets import SentencesDataset, ParallelSentencesDataset
  File "/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/sentence_transformers/datasets/__init__.py", line 1, in <module>
    from .DenoisingAutoEncoderDataset import DenoisingAutoEncoderDataset
  File "/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/sentence_transformers/datasets/DenoisingAutoEncoderDataset.py", line 1, in <module>
    from torch.utils.data import Dataset
  File "/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torch/__init__.py", line 1465, in <module>
    from . import _meta_registrations
  File "/home/viktor/min

OK, now let's build a classifier. First, let's prepare the array of proposed sentences

In [6]:
import pandas as pd

df = pd.read_csv("/home/viktor/Documents/kaggle/kaggle_llm/data/wiki-sci-2-w-sentence-context/train_1k_w_context.csv")

def get_context(text):
    x = text.split(" ### ")
    # remove empty strings
    x = [x for x in x if len(x) > 0]
    
    assert len(x) == 2, f"Unsuccesful prompt splitting . len(x) = {len(x)}, x={x}"
    return x[0]


df['context'] = df['prompt'].apply(lambda x: get_context(x))

def get_prompt(text):
    x = text.split(" ### ")
    # remove empty strings
    x = [x for x in x if len(x) > 0]
    assert len(x) == 2, f"Unsuccesful prompt splitting . len(x) = {len(x)}"
    return x[1]

df['prompt'] = df['prompt'].apply(lambda x: get_prompt(x))


df['context_sentences'] = df['context'].apply(lambda x: x.split(". "))
df['context_sentences_len'] = df['context_sentences'].apply(lambda x: len(x))
df['text'] = df[['prompt', 'context_sentences', 'A', 'B', 'C', 'D', 'E']].apply(lambda x: "prompt: " + x[0] + "\n ### context: " + " ".join(x[1]) + "\n ### A: " + x[2] + "\n ### B: " + x[3] + "\n ### C: " + x[4] + "\n ### D: " + x[5] + "\n ### E: " + x[6], axis=1)

df

,prompt,A,B,C,D,E,answer,context,context_sentences,context_sentences_len,text
0,What is the main advantage of having larger a...,Enhanced physical performance,Greater protection against injury,Higher aesthetic appeal,Improved balance and stability,Increased pleasure during sexual intercourse,B,"Context: In the case of humans, females tend t...","[Context: In the case of humans, females tend ...",62,prompt: What is the main advantage of having ...
1,What is the approximate amplitude of the shor...,0.001 magnitudes,0.009 magnitudes,0.005 magnitudes,0.015 magnitudes,0.02 magnitudes,B,Context: The amplitude can be up to 0.6 magnit...,[Context: The amplitude can be up to 0.6 magni...,62,prompt: What is the approximate amplitude of ...
2,What is the commutator in the context of deri...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a linear endomorphism of an ...,E,Context: \n DerK(A) is a Lie algebra with Lie...,[Context: \n DerK(A) is a Lie algebra with Li...,62,prompt: What is the commutator in the context...
3,What is the main goal of diabetes management ...,To eliminate the need for insulin injections i...,To prevent the development of cardiovascular d...,To avoid acute problems of hyperglycemia or hy...,To maintain blood sugar levels as close to nor...,To restore pancreatic beta cell function in ty...,D,Context: Some degree of insulin resistance may...,[Context: Some degree of insulin resistance ma...,62,prompt: What is the main goal of diabetes man...
4,Which of the following statements is true abo...,It is a benign nodular thickening of the vas d...,It is a low-grade adenocarcinoma,It is a complication experienced in approximat...,It is a form of sterilization for males,It is a male genital disorder,C,Context: Vasitis nodosa is a complication expe...,[Context: Vasitis nodosa is a complication exp...,61,prompt: Which of the following statements is ...
...,...,...,...,...,...,...,...,...,...,...,...
95,What is the main condition that Euler system ...,They must be indexed by different number fields.,They must be congruent to each other modulo a ...,They must have special values related to Euler...,They must be elements of a specific Galois coh...,They must be related by a simple formula for d...,E,"Context: In mathematics, an Euler system is a ...","[Context: In mathematics, an Euler system is a...",47,prompt: What is the main condition that Euler...
96,What is the role of coefficients in a formal ...,To determine the radius of convergence of the ...,To indicate the position of the terms in the s...,To define the function represented by the series.,To represent the value of the variable.,To determine the number of terms in the series.,B,"Context: Thus, the series may no longer repres...","[Context: Thus, the series may no longer repre...",45,prompt: What is the role of coefficients in a...
97,What is the relationship between PSA levels a...,A PSA level of 2.0ng/mL above the nadir is the...,PSA levels should decrease gradually after rad...,PSA levels may continue to decrease for severa...,A rise in PSA levels above the nadir indicates...,PSA levels are not reliable indicators of pros...,D,Context: A subsequent increase in PSA levels b...,[Context: A subsequent increase in PSA levels ...,62,prompt: What is the relationship between PSA ...
98,What is the role of Mn(III) ions produced by ...,To regulate the transcriptional activity of Mn...,To stabilize the heme cofactor in the enzyme,To catalyze the oxidation of phenolic compound...,To bind to the Fe(III) ion in the heme cofactor,To facilitate the release of Mn(III)-carboxyli...,C,Context: The Mn3+ ion itself can degrade ligni...,[Context: The Mn3+ ion itself can degrade lign...,52,prompt: What is the role of Mn(III) ions prod...


In [13]:
df_2 = df.explode('context_sentences').reset_index(drop=True)
df_2['context_sentences_len'] = df_2['context_sentences'].apply(lambda x: len(x.split(" ")))
import matplotlib.pyplot as plt



In [20]:
df_2['context_sentences_len'].min(), df_2['context_sentences_len'].max()

(1, 1056)

In [3]:
df.iloc[0]['text']

'prompt:  What is the main advantage of having larger and thicker buttocks in humans?\n ### context: Context: In the case of humans, females tend to have proportionally wider and thicker buttocks due to higher subcutaneous fat and proportionally wider hips The two main advantages are felt to be bipedal locomotion and greater thermal load dissipation capacity due to better sweating and less hair.\n\nSexual selection\n\nMarkus J \n\nIn many cultures, the buttocks play a role in sexual attraction Augmentation with a gluteal implant is the method most effective for enlarging the buttocks of the man or of the woman whose body possesses few stores of excess adipose fat in the lower portion of the trunk, the buttocks and thighs, the anatomic regions where the human body usually stores excess body fat It is one of the major defining features of the buttocks This transference of weight contributes to energy conservation during locomotion.\n\nKnee\n\nHuman knee joints are enlarged for the same r

In [4]:
df = df[:10]
df

,prompt,A,B,C,D,E,answer,context,context_sentences,context_sentences_len,text
0,What is the main advantage of having larger a...,Enhanced physical performance,Greater protection against injury,Higher aesthetic appeal,Improved balance and stability,Increased pleasure during sexual intercourse,B,"Context: In the case of humans, females tend t...","[Context: In the case of humans, females tend ...",62,prompt: What is the main advantage of having ...
1,What is the approximate amplitude of the shor...,0.001 magnitudes,0.009 magnitudes,0.005 magnitudes,0.015 magnitudes,0.02 magnitudes,B,Context: The amplitude can be up to 0.6 magnit...,[Context: The amplitude can be up to 0.6 magni...,62,prompt: What is the approximate amplitude of ...
2,What is the commutator in the context of deri...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a function that maps an alge...,The commutator is a linear endomorphism of an ...,E,Context: \n DerK(A) is a Lie algebra with Lie...,[Context: \n DerK(A) is a Lie algebra with Li...,62,prompt: What is the commutator in the context...
3,What is the main goal of diabetes management ...,To eliminate the need for insulin injections i...,To prevent the development of cardiovascular d...,To avoid acute problems of hyperglycemia or hy...,To maintain blood sugar levels as close to nor...,To restore pancreatic beta cell function in ty...,D,Context: Some degree of insulin resistance may...,[Context: Some degree of insulin resistance ma...,62,prompt: What is the main goal of diabetes man...
4,Which of the following statements is true abo...,It is a benign nodular thickening of the vas d...,It is a low-grade adenocarcinoma,It is a complication experienced in approximat...,It is a form of sterilization for males,It is a male genital disorder,C,Context: Vasitis nodosa is a complication expe...,[Context: Vasitis nodosa is a complication exp...,61,prompt: Which of the following statements is ...
5,"What is the role of Mannan 1,2-(1,3)-alpha-ma...",It acts as a catalyst in the synthesis of alph...,It acts as a catalyst in the breakdown of alph...,It acts as a regulator of glucose metabolism i...,It acts as a signaling molecule in the yeast m...,It acts as a transporter for mannose in yeast ...,B,Context: This enzyme catalyses the following c...,[Context: This enzyme catalyses the following ...,61,"prompt: What is the role of Mannan 1,2-(1,3)-..."
6,What is the main focus of the study of vinylo...,Their impact on the activity of tannase enzyme.,Their ability to form esters with benzoic acid.,Their role in the synthesis of pyrogallols.,Their significance in the metabolism of digall...,Their interaction with gallotannins in Pistaci...,C,Context: This enzyme can also be used to produ...,[Context: This enzyme can also be used to prod...,62,prompt: What is the main focus of the study o...
7,What is calcination?,A method to reduce metal oxides,A process to form alloys,Thermal decomposition of a material,A technique to remove impurities,A reaction between metals and acids,C,Context: Calcination refers to thermal treatme...,[Context: Calcination refers to thermal treatm...,62,prompt: What is calcination?\n ### context: C...
8,What is the purpose of filling in the table i...,To assign scores to every possible alignment.,To find all possible alignments having the hig...,To calculate the edit distance between two str...,To divide a large problem into a series of sma...,To construct a grid for protein or nucleotide ...,A,Context: Since the algorithm fills an table t...,[Context: Since the algorithm fills an table ...,46,prompt: What is the purpose of filling in the...
9,Who discovered LFA-1?,Timothy Springer,Louis Pasteur,Robert Koch,Paul Ehrlich,Alexander Fleming,A,"Context: LFA-1, a leukocyte integrin, was firs...","[Context: LFA-1, a leukocyte integrin, was fir...",62,prompt: Who discovered LFA-1?\n ### context: 

In [ ]:
# import gradio as gr
# import pandas as pd
# import logging

# # Enable logging
# logging.basicConfig(level=logging.INFO)

# idx2label = {}
# current_sentence_idx = [0]

# def save_results():
#     result_df = pd.DataFrame(list(idx2label.items()), columns=['index', 'label'])
#     result_df[['row_index', 'sentence_index']] = pd.DataFrame(result_df['index'].tolist(), index=result_df.index)
#     result_df.drop(columns='index', inplace=True)
#     result_df.to_csv("results.csv", index=False)
#     logging.info("Results saved to results.csv")


# def create_output_string(row_idx, sentence_idx):
#     row = df.iloc[row_idx]
#     text = f"Prompt: {row['prompt']}\n"
#     text += f"A) {row['A']}\nB) {row['B']}\nC) {row['C']}\nD) {row['D']}\nE) {row['E']}\n"
#     text += f"Answer: {row['answer']}\n"
#     text += f"Context Sentence: {row['context_sentences'][sentence_idx]}"
#     return text



# def move_to_next(row_idx):
#     if current_sentence_idx[0] < len(df.iloc[row_idx]['context_sentences']) - 1:
#         current_sentence_idx[0] += 1
#     else:
#         if row_idx < len(df) - 1:
#             current_sentence_idx[0] = 0
#             return row_idx + 1
#         else:
#             return None
#     return row_idx


# def prev_fn(row_idx):
#     prev_idx = move_to_previous(row_idx)
#     if prev_idx is None:
#         return row_idx, "Start of data reached.", "Start of data"
    
#     return prev_idx, create_output_string(prev_idx, current_sentence_idx[0]), idx2label.get((prev_idx, current_sentence_idx[0]), "unlabeled")


# def move_to_previous(row_idx):
#     if current_sentence_idx[0] > 0:
#         current_sentence_idx[0] -= 1
#     else:
#         if row_idx > 0:
#             row_idx -= 1
#             current_sentence_idx[0] = len(df.iloc[row_idx]['context_sentences']) - 1
#         else:
#             return None
#     return row_idx


# def label_as_relevant(row_idx):
#     global idx2label
#     idx2label[(row_idx, current_sentence_idx[0])] = "Relevant"
#     new_idx = move_to_next(row_idx)
    
#     if new_idx is None:
#         save_results()
#         return row_idx, "All data labeled. Results saved.", "End of data"
    
#     return new_idx, create_output_string(new_idx, current_sentence_idx[0]), idx2label.get((new_idx, current_sentence_idx[0]), "unlabeled")

# def label_as_not_relevant(row_idx):
#     global idx2label
#     idx2label[(row_idx, current_sentence_idx[0])] = "Not Relevant"
#     new_idx = move_to_next(row_idx)
    
#     if new_idx is None:
#         save_results()
#         return row_idx, "All data labeled. Results saved.", "End of data"
    
#     return new_idx, create_output_string(new_idx, current_sentence_idx[0]), idx2label.get((new_idx, current_sentence_idx[0]), "unlabeled")


# def next_fn(row_idx):
#     next_idx = move_to_next(row_idx)
#     if next_idx is None:
#         return row_idx, "End of data reached.", "End of data"
    
#     return next_idx, create_output_string(next_idx, current_sentence_idx[0]), idx2label.get((next_idx, current_sentence_idx[0]), "unlabeled")


# with gr.Blocks() as demo:
#     starting_idx = 0
#     with gr.Row():
#         with gr.Column():
#             idx = gr.Slider(value=starting_idx, label="idx")
#             text = gr.Textbox(value=create_output_string(starting_idx, current_sentence_idx[0]), label="text")
#         with gr.Column():
#             with gr.Row():
#                 relevant_btn = gr.Button(value="Relevant")
#                 not_relevant_btn = gr.Button(value="Not Relevant")
#             with gr.Row():
#                 previous = gr.Button(value="Previous example")
#                 next_ = gr.Button(value="Next example")
#                 label = gr.Textbox(value=idx2label.get((starting_idx, current_sentence_idx[0]), "unlabeled"), label="Label")
    
#     previous.click(fn=prev_fn, inputs=idx, outputs=[idx, text, label])
#     next_.click(fn=next_fn, inputs=idx, outputs=[idx, text, label])
#     relevant_btn.click(fn=label_as_relevant, inputs=[idx], outputs=[idx, text, label])
#     not_relevant_btn.click(fn=label_as_not_relevant, inputs=[idx], outputs=[idx, text, label])

# demo.launch(debug=True, show_error=True, share=True)
